In [ ]:
import re

import benchlingapi
import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import toml
from cytoolz import reduce
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Detect cutters

In [ ]:
# TODO: avoid cutting in the interesting part of our sequences (e.g., parts+GG sites), if possible

In [ ]:
plib_plasmids = reg.get_df(("pLIB", "plasmids"))

In [ ]:
plib_plasmids

In [ ]:
plib_plasmids["Tags"].value_counts()

In [ ]:
plib_plasmids[plib_plasmids["Tags"].str.contains("sequestration")]

In [ ]:
plasmids = {f"pLIB{num}": reg.get(f"pLIB{num}")["_seq"] for num in range(219, 223)}

In [ ]:
single_cutters = {
    name: {
        e
        for e, cuts in Restriction.CommOnly.search(plasmid.seq).items()
        if len(cuts) == 1
    }
    for name, plasmid in plasmids.items()
}

In [ ]:
reduce(set.intersection, single_cutters.values())

In [ ]:
single_cutters["pLIB219"] & single_cutters["pLIB220"]

In [ ]:
single_cutters["pLIB219"] & single_cutters["pLIB222"]

In [ ]:
single_cutters["pLIB221"] & single_cutters["pLIB222"]

In [ ]:
Restriction.Restriction_Dictionary.rest_dict

In [ ]:
x

# Find fewest cutters